# AlgoSpace Advanced Data Preparation Pipeline

This notebook implements a comprehensive feature engineering pipeline for the AlgoSpace MARL trading system.

## Key Features:
- ES futures data processing with Heiken Ashi transformation
- Advanced LVN (Low Volume Node) strength scoring
- MMD (Maximum Mean Discrepancy) feature vector calculation for the Regime Detection Engine
- Interaction feature engineering
- Separate output files for main MARL and RDE training

## Architecture Context:
This data preparation supports a hierarchical system with three main models:
1. **Regime Detection Engine (RDE):** Uses MMD feature vectors to learn market regimes
2. **Risk Management Sub-system (M-RMS):** Uses trade plan features
3. **Main MARL Core:** Uses all features for final trading decisions

## 1. Environment Setup

In [ ]:
# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("✅ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("⚠️ Not running in Google Colab")

# Mount Google Drive
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_BASE = "/content/drive/MyDrive/AlgoSpace"
    !mkdir -p {DRIVE_BASE}/data/{raw,processed,compressed}
else:
    DRIVE_BASE = "./drive_simulation"
    import os
    os.makedirs(f"{DRIVE_BASE}/data/raw", exist_ok=True)
    os.makedirs(f"{DRIVE_BASE}/data/processed", exist_ok=True)

In [ ]:
# Install required packages
!pip install -q yfinance pandas numpy h5py pyarrow
!pip install -q ta pandas-ta
!pip install -q scikit-learn tqdm
!pip install -q signatory  # For path signatures
!pip install -q pot  # For MMD calculations
!pip install -q scipy matplotlib seaborn

print("✅ Packages installed")

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import yfinance as yf
import h5py
from datetime import datetime, timedelta
import ta
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import json
import warnings
warnings.filterwarnings('ignore')

# Advanced feature engineering imports
from scipy import stats
from scipy.spatial.distance import cdist
import ot  # Optimal transport for MMD
from collections import deque

print("✅ Libraries imported")

## 2. Data Configuration

In [ ]:
# Data configuration for ES futures
DATA_CONFIG = {
    # ES futures symbol (using SPY as proxy for demonstration)
    'symbol': 'ES=F',  # E-mini S&P 500 futures
    'proxy_symbol': 'SPY',  # Use SPY if ES futures data not available
    
    # Date range (5 years)
    'start_date': '2019-01-01',
    'end_date': '2023-12-31',
    
    # Data splits
    'train_ratio': 0.7,
    'val_ratio': 0.15,
    'test_ratio': 0.15,
    
    # Heiken Ashi parameters
    'ha_timeframe': '30min',
    
    # LVN parameters
    'volume_profile_bins': 50,
    'lvn_lookback_days': 20,
    'lvn_threshold_percentile': 30,  # Bottom 30% of volume nodes
    
    # MMD parameters
    'mmd_window_size': 96,  # 96 * 30min = 48 hours
    'n_market_regimes': 7,
    'path_signature_depth': 3,
    
    # Feature engineering
    'lookback_periods': [5, 10, 20, 50, 100, 200],
    'technical_indicators': True,
    'market_microstructure': True,
    
    # Normalization
    'normalization_method': 'robust',
    'clip_outliers': True,
    'outlier_threshold': 5
}

print("📋 Data Configuration:")
print(f"- Symbol: {DATA_CONFIG['symbol']} (or {DATA_CONFIG['proxy_symbol']} as proxy)")
print(f"- Date Range: {DATA_CONFIG['start_date']} to {DATA_CONFIG['end_date']}")
print(f"- Heiken Ashi Timeframe: {DATA_CONFIG['ha_timeframe']}")
print(f"- Market Regimes: {DATA_CONFIG['n_market_regimes']}")
print(f"- MMD Window Size: {DATA_CONFIG['mmd_window_size']} bars")

## 3. Download Market Data

In [ ]:
# Download ES futures data
def download_es_futures_data(config):
    """Download ES futures data or use proxy if not available."""
    
    print(f"📥 Downloading ES futures data...")
    
    try:
        # Try to download ES futures data
        ticker = yf.Ticker(config['symbol'])
        data = ticker.history(start=config['start_date'], end=config['end_date'], interval='30m')
        
        if len(data) > 0:
            print(f"✅ Downloaded {config['symbol']} data: {len(data)} rows")
            data_symbol = config['symbol']
        else:
            raise ValueError("No ES futures data available")
            
    except Exception as e:
        print(f"⚠️ ES futures data not available: {e}")
        print(f"📥 Using {config['proxy_symbol']} as proxy...")
        
        # Use proxy data
        ticker = yf.Ticker(config['proxy_symbol'])
        data = ticker.history(start=config['start_date'], end=config['end_date'], interval='30m')
        data_symbol = config['proxy_symbol']
        
        if len(data) == 0:
            raise ValueError("No data available for proxy symbol either")
        
        print(f"✅ Downloaded {config['proxy_symbol']} proxy data: {len(data)} rows")
    
    # Clean data
    data = data.dropna()
    data = data[data['Volume'] > 0]
    
    # Forward fill any remaining gaps
    data = data.fillna(method='ffill')
    
    return data, data_symbol

# Download the data
es_data, used_symbol = download_es_futures_data(DATA_CONFIG)

print(f"\n📊 Data Overview:")
print(f"  - Symbol Used: {used_symbol}")
print(f"  - Shape: {es_data.shape}")
print(f"  - Date Range: {es_data.index[0]} to {es_data.index[-1]}")
print(f"  - Columns: {list(es_data.columns)}")

In [ ]:
# Calculate Heiken Ashi data
def calculate_heiken_ashi(df):
    """Calculate Heiken Ashi candles from OHLC data."""
    
    ha_df = pd.DataFrame(index=df.index)
    
    # Calculate Heiken Ashi values
    ha_df['HA_Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    
    # Initialize first HA_Open
    ha_df['HA_Open'] = 0.0
    ha_df.iloc[0, ha_df.columns.get_loc('HA_Open')] = (df['Open'].iloc[0] + df['Close'].iloc[0]) / 2
    
    # Calculate subsequent HA_Open values
    for i in range(1, len(ha_df)):
        ha_df.iloc[i, ha_df.columns.get_loc('HA_Open')] = (
            ha_df.iloc[i-1, ha_df.columns.get_loc('HA_Open')] + 
            ha_df.iloc[i-1, ha_df.columns.get_loc('HA_Close')]
        ) / 2
    
    # Calculate HA High and Low
    ha_df['HA_High'] = pd.concat([df['High'], ha_df['HA_Open'], ha_df['HA_Close']], axis=1).max(axis=1)
    ha_df['HA_Low'] = pd.concat([df['Low'], ha_df['HA_Open'], ha_df['HA_Close']], axis=1).min(axis=1)
    
    # Add volume (same as original)
    ha_df['HA_Volume'] = df['Volume']
    
    # Add derived features
    ha_df['HA_Body'] = ha_df['HA_Close'] - ha_df['HA_Open']
    ha_df['HA_UpperShadow'] = ha_df['HA_High'] - pd.concat([ha_df['HA_Open'], ha_df['HA_Close']], axis=1).max(axis=1)
    ha_df['HA_LowerShadow'] = pd.concat([ha_df['HA_Open'], ha_df['HA_Close']], axis=1).min(axis=1) - ha_df['HA_Low']
    ha_df['HA_Direction'] = np.sign(ha_df['HA_Body'])
    
    return ha_df

# Calculate Heiken Ashi
ha_data = calculate_heiken_ashi(es_data)

# Combine original and HA data
combined_data = pd.concat([es_data, ha_data], axis=1)

print("✅ Heiken Ashi data calculated")
print(f"   Combined data shape: {combined_data.shape}")
print(f"   HA columns: {[col for col in ha_data.columns]}")

In [ ]:
# Advanced LVN (Low Volume Node) calculation and strength scoring
def identify_lvns_with_strength(data, config):
    """Identify LVNs from volume profile and calculate strength scores."""
    
    print("🔍 Identifying LVNs and calculating strength scores...")
    
    lvn_data = []
    lookback = config['lvn_lookback_days'] * 48  # Convert days to 30-min bars
    
    for i in tqdm(range(lookback, len(data)), desc="Processing LVNs"):
        # Get window data
        window = data.iloc[i-lookback:i]
        
        # Create volume profile
        price_min = window['HA_Low'].min()
        price_max = window['HA_High'].max()
        bins = np.linspace(price_min, price_max, config['volume_profile_bins'] + 1)
        
        # Calculate volume at each price level
        volume_profile = np.zeros(config['volume_profile_bins'])
        
        for idx, row in window.iterrows():
            # Distribute volume across the bar's range
            bar_low_bin = np.searchsorted(bins, row['HA_Low'], side='left')
            bar_high_bin = np.searchsorted(bins, row['HA_High'], side='right')
            
            if bar_high_bin > bar_low_bin:
                volume_per_bin = row['HA_Volume'] / (bar_high_bin - bar_low_bin)
                for bin_idx in range(max(0, bar_low_bin), min(config['volume_profile_bins'], bar_high_bin)):
                    volume_profile[bin_idx] += volume_per_bin
        
        # Identify LVNs (low volume nodes)
        threshold = np.percentile(volume_profile, config['lvn_threshold_percentile'])
        lvn_indices = np.where(volume_profile < threshold)[0]
        
        # Calculate LVN levels and strength scores
        current_lvns = []
        for lvn_idx in lvn_indices:
            lvn_price = (bins[lvn_idx] + bins[lvn_idx + 1]) / 2
            strength_score = calculate_lvn_strength(
                lvn_price, 
                window, 
                data.iloc[i:min(i+48, len(data))],  # Look ahead 1 day for strength
                volume_profile[lvn_idx],
                np.mean(volume_profile)
            )
            current_lvns.append({
                'price': lvn_price,
                'strength': strength_score,
                'volume_ratio': volume_profile[lvn_idx] / (np.mean(volume_profile) + 1e-10)
            })
        
        # Sort by strength and keep top LVNs
        current_lvns.sort(key=lambda x: x['strength'], reverse=True)
        
        lvn_data.append({
            'timestamp': data.index[i],
            'lvns': current_lvns[:5],  # Keep top 5 LVNs
            'strongest_lvn_price': current_lvns[0]['price'] if current_lvns else np.nan,
            'strongest_lvn_strength': current_lvns[0]['strength'] if current_lvns else 0,
            'n_lvns': len(current_lvns)
        })
    
    return pd.DataFrame(lvn_data).set_index('timestamp')


def calculate_lvn_strength(level, historical_window, future_window, lvn_volume, avg_volume):
    """
    Calculate LVN strength score (0-100) based on:
    - Number of tests
    - Magnitude of bounces
    - Recency
    - Volume characteristics
    """
    
    strength_components = []
    
    # 1. Test frequency score (how many times price tested this level)
    tolerance = 0.001  # 0.1% tolerance
    tests = 0
    bounces = []
    
    for idx, row in historical_window.iterrows():
        if abs(row['HA_Low'] - level) / level < tolerance or abs(row['HA_High'] - level) / level < tolerance:
            tests += 1
            # Calculate bounce magnitude
            next_bars = historical_window.loc[idx:].iloc[1:6]  # Next 5 bars
            if len(next_bars) > 0:
                bounce_magnitude = abs(next_bars['HA_Close'].iloc[-1] - row['HA_Close']) / row['HA_Close']
                bounces.append(bounce_magnitude)
    
    test_score = min(tests * 10, 30)  # Max 30 points for tests
    
    # 2. Bounce magnitude score
    if bounces:
        avg_bounce = np.mean(bounces) * 100  # Convert to percentage
        bounce_score = min(avg_bounce * 5, 25)  # Max 25 points for bounces
    else:
        bounce_score = 0
    
    # 3. Recency score (more recent tests score higher)
    recency_weights = np.linspace(0.5, 1.0, len(historical_window))
    recent_tests = 0
    
    for i, (idx, row) in enumerate(historical_window.iterrows()):
        if abs(row['HA_Low'] - level) / level < tolerance or abs(row['HA_High'] - level) / level < tolerance:
            recent_tests += recency_weights[i]
    
    recency_score = min(recent_tests * 5, 20)  # Max 20 points for recency
    
    # 4. Volume void score (lower volume = stronger LVN)
    volume_void_score = (1 - lvn_volume / avg_volume) * 15  # Max 15 points
    
    # 5. Future validation score (did the level hold in the near future?)
    future_score = 0
    if len(future_window) > 0:
        future_tests = 0
        for idx, row in future_window.iterrows():
            if abs(row['HA_Low'] - level) / level < tolerance:
                future_tests += 1
                if row['HA_Close'] > level:  # Bounce up from level
                    future_score += 2
        
        future_score = min(future_score, 10)  # Max 10 points
    
    # Calculate total strength score
    total_score = test_score + bounce_score + recency_score + volume_void_score + future_score
    
    # Normalize to 0-100
    return min(total_score, 100)


# Calculate LVNs with strength scores
lvn_df = identify_lvns_with_strength(combined_data, DATA_CONFIG)

# Merge with main data
combined_data = combined_data.join(lvn_df[['strongest_lvn_price', 'strongest_lvn_strength', 'n_lvns']], how='left')
combined_data.fillna(method='ffill', inplace=True)

print("✅ LVN analysis complete")
print(f"   LVN columns added: {['strongest_lvn_price', 'strongest_lvn_strength', 'n_lvns']}")

In [ ]:
# MMD Feature Vector Calculation for RDE
def identify_market_regimes(data, n_regimes=7):
    """Identify archetypal market regimes using unsupervised clustering."""
    
    print("🎯 Identifying market regimes...")
    
    # Extract features for clustering
    features = []
    window_size = 48  # 1 day of 30-min bars
    
    for i in range(window_size, len(data)):
        window = data.iloc[i-window_size:i]
        
        # Calculate regime features
        regime_features = {
            'volatility': window['HA_Close'].pct_change().std() * np.sqrt(48 * 252),  # Annualized
            'momentum': (window['HA_Close'].iloc[-1] / window['HA_Close'].iloc[0] - 1),
            'volume_trend': window['HA_Volume'].mean() / window['HA_Volume'].iloc[:24].mean(),
            'range_ratio': (window['HA_High'].max() - window['HA_Low'].min()) / window['HA_Close'].mean(),
            'direction_consistency': window['HA_Direction'].mean(),
            'shadow_ratio': (window['HA_UpperShadow'] + window['HA_LowerShadow']).mean() / window['HA_Body'].abs().mean()
        }
        
        features.append(list(regime_features.values()))
    
    features = np.array(features)
    
    # Normalize features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Fit Gaussian Mixture Model
    gmm = GaussianMixture(n_components=n_regimes, covariance_type='full', random_state=42)
    gmm.fit(features_scaled)
    
    # Get regime assignments
    regime_labels = gmm.predict(features_scaled)
    
    # Calculate regime centers (reference signatures)
    regime_centers = gmm.means_
    
    print(f"✅ Identified {n_regimes} market regimes")
    
    # Show regime distribution
    unique, counts = np.unique(regime_labels, return_counts=True)
    for regime, count in zip(unique, counts):
        print(f"   Regime {regime}: {count} samples ({count/len(regime_labels)*100:.1f}%)")
    
    return gmm, scaler, regime_centers, regime_labels


def calculate_path_signature(window_data, depth=3):
    """Calculate path signature features for a data window."""
    
    # Extract relevant price/volume paths
    paths = np.column_stack([
        window_data['HA_Close'].values,
        window_data['HA_Volume'].values,
        window_data['HA_Body'].values
    ])
    
    # Normalize paths
    paths_normalized = (paths - paths.mean(axis=0)) / (paths.std(axis=0) + 1e-10)
    
    # Simple signature approximation (without signatory library)
    signature_features = []
    
    # Level 1: increments
    increments = np.diff(paths_normalized, axis=0)
    signature_features.extend(increments.mean(axis=0))
    signature_features.extend(increments.std(axis=0))
    
    # Level 2: areas (simplified)
    if depth >= 2:
        for i in range(paths_normalized.shape[1]):
            for j in range(i, paths_normalized.shape[1]):
                area = np.sum(paths_normalized[:-1, i] * np.diff(paths_normalized[:, j]))
                signature_features.append(area)
    
    # Level 3: volumes (simplified)
    if depth >= 3:
        for i in range(min(3, paths_normalized.shape[1])):
            volume = np.sum(paths_normalized[:-2, i] * np.diff(paths_normalized[:-1, i]) * np.diff(paths_normalized[1:, i]))
            signature_features.append(volume)
    
    return np.array(signature_features)


def calculate_mmd_features(data, regime_centers, scaler, config):
    """Calculate MMD feature vectors for each time window."""
    
    print("📊 Calculating MMD feature vectors...")
    
    mmd_features_list = []
    window_size = config['mmd_window_size']
    
    # Identify regimes and calculate reference signatures
    gmm, regime_scaler, regime_centers, _ = identify_market_regimes(data, config['n_market_regimes'])
    
    # Calculate reference path signatures for each regime
    reference_signatures = {}
    regime_windows = {i: [] for i in range(config['n_market_regimes'])}
    
    # Group windows by regime
    for i in tqdm(range(window_size, len(data)), desc="Grouping by regime"):
        window = data.iloc[i-window_size:i]
        
        # Extract features for regime classification
        regime_features = np.array([
            window['HA_Close'].pct_change().std() * np.sqrt(48 * 252),
            (window['HA_Close'].iloc[-1] / window['HA_Close'].iloc[0] - 1),
            window['HA_Volume'].mean() / window['HA_Volume'].iloc[:48].mean(),
            (window['HA_High'].max() - window['HA_Low'].min()) / window['HA_Close'].mean(),
            window['HA_Direction'].mean(),
            (window['HA_UpperShadow'] + window['HA_LowerShadow']).mean() / (window['HA_Body'].abs().mean() + 1e-10)
        ]).reshape(1, -1)
        
        regime = gmm.predict(regime_scaler.transform(regime_features))[0]
        regime_windows[regime].append(window)
    
    # Calculate reference signatures
    for regime, windows in regime_windows.items():
        if windows:
            # Use median window as reference
            signatures = [calculate_path_signature(w, config['path_signature_depth']) for w in windows[:50]]
            reference_signatures[regime] = np.median(signatures, axis=0)
        else:
            reference_signatures[regime] = np.zeros(10)  # Default signature
    
    # Calculate MMD features for each window
    for i in tqdm(range(window_size, len(data)), desc="Calculating MMD"):
        window = data.iloc[i-window_size:i]
        
        # Calculate window signature
        window_signature = calculate_path_signature(window, config['path_signature_depth'])
        
        # Calculate MMD scores against each reference signature
        mmd_scores = []
        for regime in range(config['n_market_regimes']):
            ref_sig = reference_signatures[regime]
            
            # Simple MMD approximation using L2 distance
            mmd = np.linalg.norm(window_signature - ref_sig)
            mmd_scores.append(mmd)
        
        # Additional statistical features
        stat_features = [
            window['HA_Close'].pct_change().std() * np.sqrt(48 * 252),  # Realized volatility
            stats.skew(window['HA_Close'].pct_change().dropna()),  # Skewness
            stats.kurtosis(window['HA_Close'].pct_change().dropna()),  # Kurtosis
            window['HA_Volume'].std() / window['HA_Volume'].mean(),  # Volume coefficient of variation
            np.corrcoef(window['HA_Close'].values[:-1], window['HA_Close'].values[1:])[0, 1],  # Autocorrelation
        ]
        
        # Combine all features into MMD feature vector
        mmd_feature_vector = np.concatenate([mmd_scores, stat_features])
        
        mmd_features_list.append({
            'timestamp': data.index[i],
            'mmd_features': mmd_feature_vector,
            'dominant_regime': np.argmin(mmd_scores)
        })
    
    return pd.DataFrame(mmd_features_list).set_index('timestamp'), reference_signatures


# Calculate MMD features
mmd_df, reference_signatures = calculate_mmd_features(combined_data, None, None, DATA_CONFIG)

print("✅ MMD feature calculation complete")
print(f"   MMD feature vector size: {len(mmd_df['mmd_features'].iloc[0])}")
print(f"   Features: {DATA_CONFIG['n_market_regimes']} MMD scores + 5 statistical features")

# Add technical indicators including MLMI and NWRQK
def add_technical_indicators(df, lookback_periods):
    """Add technical indicators to the dataframe."""
    
    # Price-based features
    df['returns'] = df['Close'].pct_change()
    df['log_returns'] = np.log(df['Close'] / df['Close'].shift(1))
    df['high_low_ratio'] = df['High'] / df['Low']
    df['close_open_ratio'] = df['Close'] / df['Open']
    
    # Heiken Ashi specific features
    df['ha_returns'] = df['HA_Close'].pct_change()
    df['ha_body_ratio'] = df['HA_Body'] / df['HA_Close']
    df['ha_shadow_imbalance'] = (df['HA_UpperShadow'] - df['HA_LowerShadow']) / (df['HA_UpperShadow'] + df['HA_LowerShadow'] + 1e-10)
    
    # Volume features
    df['volume_sma'] = df['Volume'].rolling(window=20).mean()
    df['volume_ratio'] = df['Volume'] / df['volume_sma']
    df['dollar_volume'] = df['Close'] * df['Volume']
    df['ha_volume_trend'] = df['HA_Volume'].rolling(window=10).mean() / df['HA_Volume'].rolling(window=50).mean()
    
    # Moving averages
    for period in lookback_periods:
        df[f'sma_{period}'] = df['Close'].rolling(window=period).mean()
        df[f'ema_{period}'] = df['Close'].ewm(span=period).mean()
        df[f'close_sma_{period}_ratio'] = df['Close'] / df[f'sma_{period}']
        df[f'ha_sma_{period}'] = df['HA_Close'].rolling(window=period).mean()
    
    # Volatility indicators
    df['atr'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'])
    df['ha_atr'] = ta.volatility.average_true_range(df['HA_High'], df['HA_Low'], df['HA_Close'])
    df['bb_high'], df['bb_mid'], df['bb_low'] = ta.volatility.bollinger_hband(df['Close']), \
                                                 ta.volatility.bollinger_mavg(df['Close']), \
                                                 ta.volatility.bollinger_lband(df['Close'])
    df['bb_width'] = (df['bb_high'] - df['bb_low']) / df['bb_mid']
    df['bb_position'] = (df['Close'] - df['bb_low']) / (df['bb_high'] - df['bb_low'])
    
    # Momentum indicators
    df['rsi'] = ta.momentum.rsi(df['Close'])
    df['ha_rsi'] = ta.momentum.rsi(df['HA_Close'])
    df['macd'] = ta.trend.macd(df['Close'])
    df['macd_signal'] = ta.trend.macd_signal(df['Close'])
    df['macd_diff'] = df['macd'] - df['macd_signal']
    df['stoch'] = ta.momentum.stoch(df['High'], df['Low'], df['Close'])
    
    # Trend indicators
    df['adx'] = ta.trend.adx(df['High'], df['Low'], df['Close'])
    df['cci'] = ta.trend.cci(df['High'], df['Low'], df['Close'])
    df['aroon_up'] = ta.trend.aroon_up(df['High'], df['Low'])
    df['aroon_down'] = ta.trend.aroon_down(df['High'], df['Low'])
    
    # Support/Resistance levels
    for period in [20, 50]:
        df[f'resistance_{period}'] = df['High'].rolling(window=period).max()
        df[f'support_{period}'] = df['Low'].rolling(window=period).min()
        df[f'sr_ratio_{period}'] = (df['Close'] - df[f'support_{period}']) / \
                                   (df[f'resistance_{period}'] - df[f'support_{period}'])
    
    # MLMI (Market Level Momentum Indicator) - Custom implementation
    def calculate_mlmi(data, period=14):
        """Calculate custom MLMI indicator."""
        # Momentum component
        momentum = data['Close'].diff(period) / data['Close'].shift(period)
        
        # Level component (position relative to recent range)
        high_range = data['High'].rolling(window=period).max()
        low_range = data['Low'].rolling(window=period).min()
        level = (data['Close'] - low_range) / (high_range - low_range + 1e-10)
        
        # Combine momentum and level
        mlmi = momentum * 0.6 + level * 0.4
        
        return mlmi
    
    df['mlmi'] = calculate_mlmi(df)
    
    # NWRQK (Normalized Weighted Range Quality K) - Custom implementation
    def calculate_nwrqk(data, k_period=10):
        """Calculate custom NWRQK indicator."""
        # Range quality
        true_range = pd.DataFrame({
            'hl': data['High'] - data['Low'],
            'hc': abs(data['High'] - data['Close'].shift(1)),
            'lc': abs(data['Low'] - data['Close'].shift(1))
        }).max(axis=1)
        
        # Weighted by volume
        volume_weight = data['Volume'] / data['Volume'].rolling(window=k_period).mean()
        weighted_range = true_range * volume_weight
        
        # Normalize
        nwrqk = weighted_range / weighted_range.rolling(window=k_period).mean()
        
        return nwrqk
    
    df['nwrqk'] = calculate_nwrqk(df)
    
    # Interaction features
    df['mlmi_minus_nwrqk'] = df['mlmi'] - df['nwrqk']
    df['mlmi_times_nwrqk'] = df['mlmi'] * df['nwrqk']
    df['mlmi_nwrqk_ratio'] = df['mlmi'] / (df['nwrqk'] + 1e-10)
    
    # Additional interaction features
    df['rsi_bb_interaction'] = df['rsi'] * df['bb_position']
    df['macd_adx_interaction'] = df['macd_diff'] * df['adx']
    df['volume_atr_interaction'] = df['volume_ratio'] * df['atr']
    
    return df

# Apply technical indicators
print("🔧 Adding technical indicators...")
combined_data = add_technical_indicators(combined_data, DATA_CONFIG['lookback_periods'])

print("✅ Technical indicators added")
print(f"   Total features: {len(combined_data.columns)}")
print(f"   Key indicators: MLMI, NWRQK, RSI, MACD, ADX, ATR, BB")
print(f"   Interaction features: mlmi_minus_nwrqk, mlmi_times_nwrqk, etc.")

In [ ]:
# Add technical indicators including MLMI and NWRQK
def add_technical_indicators(df, lookback_periods):
    """Add technical indicators to the dataframe."""
    
    # Price-based features
    df['returns'] = df['Close'].pct_change()
    df['log_returns'] = np.log(df['Close'] / df['Close'].shift(1))
    df['high_low_ratio'] = df['High'] / df['Low']
    df['close_open_ratio'] = df['Close'] / df['Open']
    
    # Heiken Ashi specific features
    df['ha_returns'] = df['HA_Close'].pct_change()
    df['ha_body_ratio'] = df['HA_Body'] / df['HA_Close']
    df['ha_shadow_imbalance'] = (df['HA_UpperShadow'] - df['HA_LowerShadow']) / (df['HA_UpperShadow'] + df['HA_LowerShadow'] + 1e-10)
    
    # Volume features
    df['volume_sma'] = df['Volume'].rolling(window=20).mean()
    df['volume_ratio'] = df['Volume'] / df['volume_sma']
    df['dollar_volume'] = df['Close'] * df['Volume']
    df['ha_volume_trend'] = df['HA_Volume'].rolling(window=10).mean() / df['HA_Volume'].rolling(window=50).mean()
    
    # Moving averages
    for period in lookback_periods:
        df[f'sma_{period}'] = df['Close'].rolling(window=period).mean()
        df[f'ema_{period}'] = df['Close'].ewm(span=period).mean()
        df[f'close_sma_{period}_ratio'] = df['Close'] / df[f'sma_{period}']
        df[f'ha_sma_{period}'] = df['HA_Close'].rolling(window=period).mean()
    
    # Volatility indicators
    df['atr'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'])
    df['ha_atr'] = ta.volatility.average_true_range(df['HA_High'], df['HA_Low'], df['HA_Close'])
    df['bb_high'], df['bb_mid'], df['bb_low'] = ta.volatility.bollinger_hband(df['Close']), \
                                                 ta.volatility.bollinger_mavg(df['Close']), \
                                                 ta.volatility.bollinger_lband(df['Close'])
    df['bb_width'] = (df['bb_high'] - df['bb_low']) / df['bb_mid']
    df['bb_position'] = (df['Close'] - df['bb_low']) / (df['bb_high'] - df['bb_low'])
    
    # Momentum indicators
    df['rsi'] = ta.momentum.rsi(df['Close'])
    df['ha_rsi'] = ta.momentum.rsi(df['HA_Close'])
    df['macd'] = ta.trend.macd(df['Close'])
    df['macd_signal'] = ta.trend.macd_signal(df['Close'])
    df['macd_diff'] = df['macd'] - df['macd_signal']
    df['stoch'] = ta.momentum.stoch(df['High'], df['Low'], df['Close'])
    
    # Trend indicators
    df['adx'] = ta.trend.adx(df['High'], df['Low'], df['Close'])
    df['cci'] = ta.trend.cci(df['High'], df['Low'], df['Close'])
    df['aroon_up'] = ta.trend.aroon_up(df['High'], df['Low'])
    df['aroon_down'] = ta.trend.aroon_down(df['High'], df['Low'])
    
    # Support/Resistance levels
    for period in [20, 50]:
        df[f'resistance_{period}'] = df['High'].rolling(window=period).max()
        df[f'support_{period}'] = df['Low'].rolling(window=period).min()
        df[f'sr_ratio_{period}'] = (df['Close'] - df[f'support_{period}']) / \
                                   (df[f'resistance_{period}'] - df[f'support_{period}'])
    
    # MLMI (Market Level Momentum Indicator) - Custom implementation
    def calculate_mlmi(data, period=14):
        """Calculate custom MLMI indicator."""
        # Momentum component
        momentum = data['Close'].diff(period) / data['Close'].shift(period)
        
        # Level component (position relative to recent range)
        high_range = data['High'].rolling(window=period).max()
        low_range = data['Low'].rolling(window=period).min()
        level = (data['Close'] - low_range) / (high_range - low_range + 1e-10)
        
        # Combine momentum and level
        mlmi = momentum * 0.6 + level * 0.4
        
        return mlmi
    
    df['mlmi'] = calculate_mlmi(df)
    
    # NWRQK (Normalized Weighted Range Quality K) - Custom implementation
    def calculate_nwrqk(data, k_period=10):
        """Calculate custom NWRQK indicator."""
        # Range quality
        true_range = pd.DataFrame({
            'hl': data['High'] - data['Low'],
            'hc': abs(data['High'] - data['Close'].shift(1)),
            'lc': abs(data['Low'] - data['Close'].shift(1))
        }).max(axis=1)
        
        # Weighted by volume
        volume_weight = data['Volume'] / data['Volume'].rolling(window=k_period).mean()
        weighted_range = true_range * volume_weight
        
        # Normalize
        nwrqk = weighted_range / weighted_range.rolling(window=k_period).mean()
        
        return nwrqk
    
    df['nwrqk'] = calculate_nwrqk(df)
    
    # Interaction features
    df['mlmi_minus_nwrqk'] = df['mlmi'] - df['nwrqk']
    df['mlmi_times_nwrqk'] = df['mlmi'] * df['nwrqk']
    df['mlmi_nwrqk_ratio'] = df['mlmi'] / (df['nwrqk'] + 1e-10)
    
    # Additional interaction features
    df['rsi_bb_interaction'] = df['rsi'] * df['bb_position']
    df['macd_adx_interaction'] = df['macd_diff'] * df['adx']
    df['volume_atr_interaction'] = df['volume_ratio'] * df['atr']
    
    return df

# Apply technical indicators
print("🔧 Adding technical indicators...")
combined_data = add_technical_indicators(combined_data, DATA_CONFIG['lookback_periods'])

print("✅ Technical indicators added")
print(f"   Total features: {len(combined_data.columns)}")
print(f"   Key indicators: MLMI, NWRQK, RSI, MACD, ADX, ATR, BB")
print(f"   Interaction features: mlmi_minus_nwrqk, mlmi_times_nwrqk, etc.")

In [ ]:
# Final Data Preparation and Output
def prepare_final_datasets(combined_data, mmd_df, config):
    """Prepare final datasets for main MARL and RDE training."""
    
    print("📦 Preparing final datasets...")
    
    # Align all data by timestamp
    min_timestamp = max(combined_data.index[0], mmd_df.index[0])
    max_timestamp = min(combined_data.index[-1], mmd_df.index[-1])
    
    # Filter to common timerange
    combined_data_aligned = combined_data.loc[min_timestamp:max_timestamp]
    mmd_df_aligned = mmd_df.loc[min_timestamp:max_timestamp]
    
    # Drop any remaining NaN values
    combined_data_aligned = combined_data_aligned.fillna(method='ffill').fillna(0)
    
    # Prepare main MARL training data
    main_features = [
        # Price features
        'Open', 'High', 'Low', 'Close', 'Volume',
        'HA_Open', 'HA_High', 'HA_Low', 'HA_Close', 'HA_Volume',
        'HA_Body', 'HA_UpperShadow', 'HA_LowerShadow', 'HA_Direction',
        
        # Returns and ratios
        'returns', 'log_returns', 'ha_returns', 'high_low_ratio', 'close_open_ratio',
        'ha_body_ratio', 'ha_shadow_imbalance',
        
        # Volume features
        'volume_ratio', 'dollar_volume', 'ha_volume_trend',
        
        # Technical indicators
        'rsi', 'ha_rsi', 'macd', 'macd_signal', 'macd_diff', 'stoch',
        'atr', 'ha_atr', 'bb_width', 'bb_position',
        'adx', 'cci', 'aroon_up', 'aroon_down',
        
        # Custom indicators
        'mlmi', 'nwrqk',
        
        # Interaction features
        'mlmi_minus_nwrqk', 'mlmi_times_nwrqk', 'mlmi_nwrqk_ratio',
        'rsi_bb_interaction', 'macd_adx_interaction', 'volume_atr_interaction',
        
        # LVN features
        'strongest_lvn_price', 'strongest_lvn_strength', 'n_lvns',
        
        # Moving averages and ratios
        'sma_20', 'ema_20', 'close_sma_20_ratio',
        'sma_50', 'ema_50', 'close_sma_50_ratio',
        
        # Support/Resistance
        'resistance_20', 'support_20', 'sr_ratio_20',
        'resistance_50', 'support_50', 'sr_ratio_50'
    ]
    
    # Select available features
    available_features = [f for f in main_features if f in combined_data_aligned.columns]
    main_df = combined_data_aligned[available_features].copy()
    
    # Add dominant regime from MMD analysis
    main_df['dominant_regime'] = mmd_df_aligned['dominant_regime']
    
    print(f"✅ Main MARL dataset prepared: {main_df.shape}")
    
    # Prepare RDE training data (MMD features)
    rde_data_list = []
    
    for idx, row in mmd_df_aligned.iterrows():
        rde_data_list.append({
            'timestamp': idx,
            **{f'mmd_feature_{i}': val for i, val in enumerate(row['mmd_features'])}
        })
    
    rde_df = pd.DataFrame(rde_data_list).set_index('timestamp')
    
    print(f"✅ RDE dataset prepared: {rde_df.shape}")
    
    return main_df, rde_df


# Prepare final datasets
main_df, rde_df = prepare_final_datasets(combined_data, mmd_df, DATA_CONFIG)

# Save to parquet files
output_dir = f"{DRIVE_BASE}/data/processed"
os.makedirs(output_dir, exist_ok=True)

# Save main MARL training data
main_output_path = f"{output_dir}/training_data_main.parquet"
main_df.to_parquet(main_output_path, engine='pyarrow', compression='snappy')
print(f"✅ Main training data saved to: {main_output_path}")
print(f"   Size: {os.path.getsize(main_output_path) / 1e6:.2f} MB")

# Save RDE training data
rde_output_path = f"{output_dir}/training_data_rde.parquet"
rde_df.to_parquet(rde_output_path, engine='pyarrow', compression='snappy')
print(f"✅ RDE training data saved to: {rde_output_path}")
print(f"   Size: {os.path.getsize(rde_output_path) / 1e6:.2f} MB")

# Save metadata
metadata = {
    'created_date': datetime.now().isoformat(),
    'data_config': DATA_CONFIG,
    'data_symbol': used_symbol,
    'date_range': {
        'start': str(main_df.index[0]),
        'end': str(main_df.index[-1])
    },
    'main_features': list(main_df.columns),
    'rde_features': list(rde_df.columns),
    'n_samples': len(main_df),
    'reference_signatures': {str(k): v.tolist() for k, v in reference_signatures.items()}
}

metadata_path = f"{output_dir}/data_preparation_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✅ Metadata saved to: {metadata_path}")

In [ ]:
# Create comprehensive data preparation summary
summary = f"""
# Advanced Data Preparation Summary

## Dataset Information
- **Created**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- **Symbol**: {used_symbol}
- **Date Range**: {main_df.index[0]} to {main_df.index[-1]}
- **Total Samples**: {len(main_df):,}
- **Timeframe**: 30-minute bars with Heiken Ashi

## Feature Engineering Completed
### 1. Heiken Ashi Transformation
- HA OHLC values calculated
- HA body, shadows, and direction features

### 2. LVN Analysis
- Identified Low Volume Nodes from volume profile
- Calculated strength scores (0-100) based on:
  - Number of price tests
  - Bounce magnitudes
  - Recency of tests
  - Volume characteristics
- Features: strongest_lvn_price, strongest_lvn_strength, n_lvns

### 3. MMD Feature Vectors (for RDE)
- Identified {DATA_CONFIG['n_market_regimes']} archetypal market regimes
- Calculated path signatures for each window
- MMD scores against reference regime signatures
- Additional statistical features (volatility, skew, kurtosis, etc.)
- Total MMD features: {len(rde_df.columns)}

### 4. Technical Indicators
- Standard: RSI, MACD, ADX, ATR, Bollinger Bands, etc.
- Custom: MLMI (Market Level Momentum Indicator)
- Custom: NWRQK (Normalized Weighted Range Quality K)
- Interaction features: mlmi_minus_nwrqk, mlmi_times_nwrqk, etc.

## Output Files
### 1. training_data_main.parquet
- Path: `{main_output_path}`
- Shape: {main_df.shape}
- Features: {len(main_df.columns)} columns
- Size: {os.path.getsize(main_output_path) / 1e6:.2f} MB
- Use: Main MARL training and simulation

### 2. training_data_rde.parquet
- Path: `{rde_output_path}`
- Shape: {rde_df.shape}
- Features: {len(rde_df.columns)} MMD features
- Size: {os.path.getsize(rde_output_path) / 1e6:.2f} MB
- Use: Regime Detection Engine training

## Usage in Training Notebooks
```python
# Load main MARL data
import pandas as pd
main_data = pd.read_parquet('{main_output_path}')

# Load RDE data
rde_data = pd.read_parquet('{rde_output_path}')
```

## Key Features for Each Model
### Main MARL Core
- All technical indicators and HA features
- LVN strength scores
- Interaction features (mlmi_minus_nwrqk, etc.)
- Dominant regime labels

### Regime Detection Engine (RDE)
- MMD scores for {DATA_CONFIG['n_market_regimes']} regimes
- Statistical features (volatility, skew, kurtosis)
- Path signature components

### Risk Management Sub-system
Will use subset of main features focused on:
- ATR and volatility measures
- LVN levels for stop placement
- Support/resistance ratios
"""

print(summary)

# Save summary
summary_file = f"{output_dir}/data_preparation_summary_{datetime.now().strftime('%Y%m%d')}.md"
with open(summary_file, 'w') as f:
    f.write(summary)
print(f"\n✅ Summary saved to: {summary_file}")

In [ ]:
print("\n🎉 Advanced data preparation complete!")
print("\n📋 Next steps:")
print("1. Use training_data_rde.parquet in Regime_Agent_Training.ipynb")
print("2. Use training_data_main.parquet in MARL_Training_Master_Colab.ipynb")
print("3. Both files contain aligned timestamps for synchronized training")
print("\nData files ready for the hierarchical MARL system training!")

In [ ]:
# Normalize data
def normalize_data(data, method='robust', clip_threshold=5):
    """Normalize data using specified method."""
    
    # Reshape data for normalization
    original_shape = data.shape
    if len(data.shape) == 3:
        # Reshape from (samples, assets, features) to (samples*assets, features)
        n_samples, n_assets, n_features = data.shape
        data_reshaped = data.reshape(-1, n_features)
    else:
        data_reshaped = data
    
    # Replace inf and -inf with NaN
    data_reshaped = np.where(np.isinf(data_reshaped), np.nan, data_reshaped)
    
    # Fill NaN with 0
    data_reshaped = np.nan_to_num(data_reshaped, 0)
    
    # Normalize
    if method == 'robust':
        scaler = RobustScaler()
    else:
        scaler = StandardScaler()
    
    data_normalized = scaler.fit_transform(data_reshaped)
    
    # Clip outliers
    if clip_threshold:
        data_normalized = np.clip(data_normalized, -clip_threshold, clip_threshold)
    
    # Reshape back
    if len(original_shape) == 3:
        data_normalized = data_normalized.reshape(original_shape)
    
    return data_normalized, scaler

# Normalize all data
print("🔧 Normalizing data...")

normalized_features = {}
scalers = {}

# Normalize feature matrices
for agent, matrix in feature_matrices.items():
    normalized_features[agent], scalers[agent] = normalize_data(
        matrix,
        method=DATA_CONFIG['normalization_method'],
        clip_threshold=DATA_CONFIG['outlier_threshold']
    )
    print(f"   {agent}: normalized")

# Normalize correlation matrices (already in [-1, 1] range)
normalized_features['correlation_matrices'] = corr_matrices

# Normalize volume profiles (already normalized to sum to 1)
normalized_features['volume_profiles'] = volume_matrices

print("✅ Data normalization complete")

In [ ]:
print("\n🎉 Data preparation complete!")
print("\n📋 Next steps:")
print("1. The processed data is now saved in your Google Drive")
print("2. Use the Master Training Notebook to load this data")
print("3. The data is optimized for efficient loading in Colab")
print("\nData file path for training:")
print(f"'{output_file}'")